Data model summary & relationships (quick)

        Customers (PK customer_id) — has zip prefix linking to GeoLocation.

        GeoLocation (PK geolocation_zip_code_prefix) — latitude/longitude for zip areas.

        Orders (PK order_id) — links to customer_id. Contains purchase/approved/delivery timestamps.

        Order_Items — line items per order (order_id, product_id, seller_id).

        Products (PK product_id) — product attributes; product_category_name links to Product_Category_Name_Translation.

        Sellers (PK seller_id) — seller address zip links to GeoLocation.

        Order_Payments — payment breakdown per order.

        Order_Reviews — reviews against orders.

        Leads_Qualified ↔ Leads_Closed — lead funnel (mql_id) with first-contact and won_date.

A. Basic data quality & ETL checks

In [23]:
import sqlite3
import pandas as pd

# Connect to DB
conn = sqlite3.connect("olist.sqlite")

query = """
SELECT 
  'customers' AS table_name, 
  COUNT(*) AS total_rows,
  SUM(CASE WHEN customer_id IS NULL THEN 1 ELSE 0 END) AS missing_customer_id,
  SUM(CASE WHEN customer_zip_code_prefix IS NULL THEN 1 ELSE 0 END) AS missing_zip
FROM customers;
"""

df = pd.read_sql_query(query, conn)
print(df)

conn.close()


  table_name  total_rows  missing_customer_id  missing_zip
0  customers       99441                    0            0


B. Orders & fulfillment times (core for delays)
        Goal: compute cycle times and identify slow steps.
        Key metrics:

        time_to_approve = order_approved_at - order_purchase_timestamp

        time_to_ship = order_delivered_carrier_date - order_approved_at

        time_to_customer = order_delivered_customer_date - order_delivered_carrier_date

        lead_time_vs_estimate = order_delivered_customer_date - order_estimated_delivery_date

In [ ]:
import sqlite3
import pandas as pd

# Connect to DB
conn = sqlite3.connect("olist.sqlite")

query = """
SELECT
  order_id,
  customer_id,
  order_purchase_timestamp AS purchase_ts,
  order_approved_at AS approved_ts,
  order_delivered_carrier_date AS carrier_ts,
  order_delivered_customer_date AS delivered_ts,

  -- Hours to approve
  (julianday(order_approved_at) - julianday(order_purchase_timestamp)) * 24 AS hours_to_approve,

  -- Hours to ship
  (julianday(order_delivered_carrier_date) - julianday(order_approved_at)) * 24 AS hours_to_ship,

  -- Carrier to customer
  (julianday(order_delivered_customer_date) - julianday(order_delivered_carrier_date)) * 24 AS hours_carrier_to_customer,

  -- Late vs estimated
  (julianday(order_delivered_customer_date) - julianday(order_estimated_delivery_date)) * 24 AS hours_late

FROM orders
WHERE order_delivered_customer_date IS NOT NULL
LIMIT 50;

"""

df = pd.read_sql_query(query, conn)
print(df)

# Save to CSV
df.to_csv("delivery_analysis.csv", index=False)

print("✅ Results exported to delivery_analysis.csv")

conn.close()

Interpretation: compute distributions (median, 90th percentile) for each metric; 90th percentile pinpoints tail risk.

C. Seller & product-level fulfillment performance

In [ ]:
import sqlite3
import pandas as pd

# Connect to DB
conn = sqlite3.connect("olist.sqlite")

query = """
WITH delivery_diff AS (
    SELECT 
        oi.seller_id,
        p.product_category_name,
        o.order_id,
        (julianday(o.order_delivered_customer_date) - julianday(o.order_estimated_delivery_date)) * 24 AS delay_hours
    FROM orders o
    JOIN order_items oi ON o.order_id = oi.order_id
    JOIN products p ON oi.product_id = p.product_id
    WHERE o.order_delivered_customer_date IS NOT NULL
),
median_calc AS (
    SELECT 
        seller_id,
        product_category_name,
        delay_hours,
        ROW_NUMBER() OVER (PARTITION BY seller_id, product_category_name ORDER BY delay_hours) AS rn,
        COUNT(*) OVER (PARTITION BY seller_id, product_category_name) AS cnt,
        order_id
    FROM delivery_diff
)
SELECT 
    seller_id,
    product_category_name,
    COUNT(DISTINCT order_id) AS orders_count,
    AVG(delay_hours) AS avg_delay_hours,
    -- crude median approximation
    AVG(delay_hours) FILTER (
        WHERE rn IN ((cnt + 1)/2, (cnt + 2)/2)
    ) AS median_delay_hours
FROM median_calc
GROUP BY seller_id, product_category_name
ORDER BY median_delay_hours DESC
LIMIT 50;
"""

df = pd.read_sql_query(query, conn)
print(df)

# Save to CSV
df.to_csv("Seller & product-level fulfillment performance.csv", index=False)
# print("✅ Results exported to delivery_analysis.csv")

conn.close()



Interpretation: sellers/categories with high median/mean delays are candidates for intervention.

D. Delivery bottlenecks by geography & time

In [ ]:
import sqlite3
import pandas as pd

# Connect to DB
conn = sqlite3.connect("olist.sqlite")

query = """
WITH last_mile AS (
    SELECT 
        c.customer_zip_code_prefix,
        (julianday(o.order_delivered_customer_date) - julianday(o.order_delivered_carrier_date)) * 24 AS last_mile_hours
    FROM orders o
    JOIN customers c ON o.customer_id = c.customer_id
    WHERE o.order_delivered_customer_date IS NOT NULL
),
ranked AS (
    SELECT 
        customer_zip_code_prefix,
        last_mile_hours,
        ROW_NUMBER() OVER (PARTITION BY customer_zip_code_prefix ORDER BY last_mile_hours) AS rn,
        COUNT(*) OVER (PARTITION BY customer_zip_code_prefix) AS cnt
    FROM last_mile
)
SELECT 
    customer_zip_code_prefix,
    COUNT(*) AS orders_count,
    AVG(last_mile_hours) AS avg_last_mile_hours,
    -- crude 90th percentile: pick row at position ceil(0.9 * cnt)
    MAX(last_mile_hours) FILTER (
        WHERE rn = CAST(0.9 * cnt AS INT)
              OR rn = CAST(0.9 * cnt + 1 AS INT)
    ) AS p90_last_mile_hours
FROM ranked
GROUP BY customer_zip_code_prefix
ORDER BY p90_last_mile_hours DESC
LIMIT 50;

"""

df = pd.read_sql_query(query, conn)
print(df)

# Save to CSV
df.to_csv("Delivery bottlenecks by geography & time.csv", index=False)
# print("✅ Results exported to delivery_analysis.csv")

conn.close()


Interpretation: zip prefixes with high p90 indicate problematic delivery areas.

This gives you:

        orders_count

        avg_last_mile_hours

        Approximate p90_last_mile_hours

E. Revenue-at-risk from late deliveries / predicted churn

Goal: measure revenue exposure and link late deliveries to churn (or negative reviews).
Steps:

        Flag orders that were delivered later than estimated.

        Aggregate payment_value per customer and segment customers with % of late orders.

        Model churn probability using logistic regression with features: late_delivery_rate, avg_delivery_time, avg_review_score, order_freq.

In [ ]:
import sqlite3
import pandas as pd

# Connect to DB
conn = sqlite3.connect("olist.sqlite")

query = """

WITH payments AS (
    SELECT 
        order_id,
        SUM(payment_value) AS total_payment
    FROM order_payments
    GROUP BY order_id
),
orders_with_flag AS (
    SELECT 
        o.order_id,
        o.customer_id,
        CASE 
            WHEN o.order_delivered_customer_date > o.order_estimated_delivery_date THEN 1
            ELSE 0
        END AS late_flag
    FROM orders o
    WHERE o.order_delivered_customer_date IS NOT NULL
),
reviews AS (
    SELECT 
        o.customer_id,
        AVG(r.review_score) AS avg_review
    FROM orders o
    JOIN order_reviews r ON o.order_id = r.order_id
    GROUP BY o.customer_id
)
SELECT 
    o.customer_id,
    COUNT(DISTINCT o.order_id) AS total_orders,
    SUM(o.late_flag) AS late_orders,
    SUM(p.total_payment) AS rev,
    r.avg_review,
    CAST(SUM(o.late_flag) AS FLOAT) / COUNT(DISTINCT o.order_id) AS late_rate
FROM orders_with_flag o
JOIN payments p ON o.order_id = p.order_id
LEFT JOIN reviews r ON o.customer_id = r.customer_id
GROUP BY o.customer_id
ORDER BY rev DESC
LIMIT 50;


"""

df = pd.read_sql_query(query, conn)
print(df)

# Save to CSV
df.to_csv("Revenue-at-risk from late deliveries.csv", index=False)
# print("✅ Results exported to delivery_analysis.csv")

conn.close()


Interpretation: customers with high late_rate & falling review scores are retention targets.

3) Analyze delivery delays & bottlenecks — method & actionable steps
            Approach — data-driven layered analysis:

Compute end-to-end times (purchase→approval, approval→carrier pickup, carrier→customer). Use medians and high percentiles (75th, 90th) to find tail problems.

In [46]:
import sqlite3
import pandas as pd

# Connect to DB
conn = sqlite3.connect("olist.sqlite")

query = """

WITH diffs AS (
    SELECT 
        order_id,
        -- convert julianday diffs (days) to hours (* 24)
        (julianday(order_approved_at) - julianday(order_purchase_timestamp)) * 24 AS purchase_to_approve,
        (julianday(order_delivered_carrier_date) - julianday(order_approved_at)) * 24 AS approve_to_carrier,
        (julianday(order_delivered_customer_date) - julianday(order_delivered_carrier_date)) * 24 AS carrier_to_customer
    FROM orders
    WHERE order_delivered_customer_date IS NOT NULL
),
ranked AS (
    SELECT 
        order_id,
        purchase_to_approve,
        approve_to_carrier,
        carrier_to_customer,
        ROW_NUMBER() OVER (ORDER BY purchase_to_approve) AS rn_p2a,
        ROW_NUMBER() OVER (ORDER BY approve_to_carrier) AS rn_a2c,
        ROW_NUMBER() OVER (ORDER BY carrier_to_customer) AS rn_c2cust,
        COUNT(*) OVER () AS cnt
    FROM diffs
)
SELECT 
    -- median = middle row(s)
    AVG(purchase_to_approve) FILTER (
        WHERE rn_p2a IN ((cnt + 1)/2, (cnt + 2)/2)
    ) AS median_purchase_to_approve,
    AVG(approve_to_carrier) FILTER (
        WHERE rn_a2c IN ((cnt + 1)/2, (cnt + 2)/2)
    ) AS median_approve_to_carrier,
    AVG(carrier_to_customer) FILTER (
        WHERE rn_c2cust IN ((cnt + 1)/2, (cnt + 2)/2)
    ) AS median_carrier_to_customer,

    -- 75th percentile
    MAX(purchase_to_approve) FILTER (WHERE rn_p2a = CAST(0.75 * cnt AS INT)) AS p75_purchase_to_approve,
    MAX(approve_to_carrier) FILTER (WHERE rn_a2c = CAST(0.75 * cnt AS INT)) AS p75_approve_to_carrier,
    MAX(carrier_to_customer) FILTER (WHERE rn_c2cust = CAST(0.75 * cnt AS INT)) AS p75_carrier_to_customer,

    -- 90th percentile
    MAX(purchase_to_approve) FILTER (WHERE rn_p2a = CAST(0.90 * cnt AS INT)) AS p90_purchase_to_approve,
    MAX(approve_to_carrier) FILTER (WHERE rn_a2c = CAST(0.90 * cnt AS INT)) AS p90_approve_to_carrier,
    MAX(carrier_to_customer) FILTER (WHERE rn_c2cust = CAST(0.90 * cnt AS INT)) AS p90_carrier_to_customer
FROM ranked;



"""

df = pd.read_sql_query(query, conn)
print(df)

# Save to CSV
df.to_csv("end-to-end times (purchase→approval, approval→carrier pickup, carrier→customer).csv", index=False)
# print("✅ Results exported to delivery_analysis.csv")

conn.close()

   median_purchase_to_approve  median_approve_to_carrier  \
0                    0.343333                  43.575417   

   median_carrier_to_customer  p75_purchase_to_approve  \
0                  170.394306                14.513889   

   p75_approve_to_carrier  p75_carrier_to_customer  p90_purchase_to_approve  \
0               85.798333               288.697222                34.591944   

   p90_approve_to_carrier  p90_carrier_to_customer  
0                143.7625               453.569167  


This will give you a single row summary with medians, 75th, and 90th percentile values for each segment:

        purchase → approval

        approval → carrier

        carrier → customer

Segment by dimension: seller_id, product_category, customer_zip_prefix, weekday/time-of-day, seasonal windows, payment_type, shipping_limit_date vs actual.

In [ ]:
import sqlite3
import pandas as pd

# Connect to DB
conn = sqlite3.connect("olist.sqlite")

query = """

WITH base AS (
    SELECT 
        o.order_id,
        o.customer_id,
        oi.seller_id,
        p.product_category_name,
        c.customer_zip_code_prefix,
        op.payment_type,
        -- delivery times in hours
        (julianday(o.order_approved_at) - julianday(o.order_purchase_timestamp)) * 24 AS purchase_to_approve,
        (julianday(o.order_delivered_carrier_date) - julianday(o.order_approved_at)) * 24 AS approve_to_carrier,
        (julianday(o.order_delivered_customer_date) - julianday(o.order_delivered_carrier_date)) * 24 AS carrier_to_customer,
        (julianday(o.order_delivered_customer_date) - julianday(o.order_estimated_delivery_date)) * 24 AS late_by_hours,
        strftime('%w', o.order_purchase_timestamp) AS purchase_weekday, -- 0=Sunday
        strftime('%H', o.order_purchase_timestamp) AS purchase_hour,
        strftime('%m', o.order_purchase_timestamp) AS purchase_month
    FROM orders o
    JOIN order_items oi ON o.order_id = oi.order_id
    JOIN products p ON oi.product_id = p.product_id
    JOIN customers c ON o.customer_id = c.customer_id
    JOIN order_payments op ON o.order_id = op.order_id
    WHERE o.order_delivered_customer_date IS NOT NULL
)
SELECT 
    seller_id,
    product_category_name,
    customer_zip_code_prefix,
    payment_type,
    purchase_weekday,
    purchase_hour,
    purchase_month,
    COUNT(DISTINCT order_id) AS total_orders,
    AVG(purchase_to_approve) AS avg_purchase_to_approve,
    AVG(approve_to_carrier) AS avg_approve_to_carrier,
    AVG(carrier_to_customer) AS avg_carrier_to_customer,
    AVG(late_by_hours) AS avg_lateness,
    SUM(CASE WHEN late_by_hours > 0 THEN 1 ELSE 0 END) * 1.0 / COUNT(*) AS late_rate
FROM base
GROUP BY 
    seller_id,
    product_category_name,
    customer_zip_code_prefix,
    payment_type,
    purchase_weekday,
    purchase_hour,
    purchase_month
ORDER BY avg_lateness DESC
LIMIT 100;




"""

df = pd.read_sql_query(query, conn)
print(df)

# Save to CSV
df.to_csv("Segment by dimension.csv", index=False)
# print("✅ Results exported to delivery_analysis.csv")

conn.close()

What this gives you

Dimensions (segmentation):

        seller_id

        product_category_name

        customer_zip_code_prefix

        payment_type

        purchase_weekday (day of week)

        purchase_hour (time of day)

        purchase_month (seasonality)

        Metrics:

        Avg time from purchase → approval, approval → carrier, carrier → customer

        Avg lateness vs. estimated

        % of late deliveries (late_rate)

        Avg allowed shipping window

Root-cause slicing:

    If time_to_approve high → internal processing issue (fraud checks, payment verification).

In [49]:
import sqlite3
import pandas as pd

# Connect to DB
conn = sqlite3.connect("olist.sqlite")

query = """

-- Time from purchase to approval by payment type
WITH approve_times AS (
    SELECT 
        o.order_id,
        CAST((julianday(o.order_approved_at) - julianday(o.order_purchase_timestamp)) * 24 * 60 AS INTEGER) AS time_to_approve_min,
        p.payment_type
    FROM orders o
    JOIN order_payments p ON o.order_id = p.order_id
    WHERE o.order_approved_at IS NOT NULL
)
SELECT 
    payment_type,
    COUNT(*) AS orders,
    ROUND(AVG(time_to_approve_min), 2) AS avg_approve_min,
    ROUND(MIN(time_to_approve_min), 2) AS min_approve,
    ROUND(MAX(time_to_approve_min), 2) AS max_approve
FROM approve_times
GROUP BY payment_type
ORDER BY avg_approve_min DESC;


"""

df = pd.read_sql_query(query, conn)
print(df)

# Save to CSV
# df.to_csv("Segment by dimension.csv", index=False)
# print("✅ Results exported to delivery_analysis.csv")

conn.close()

  payment_type  orders  avg_approve_min  min_approve  max_approve
0       boleto   19754          1986.99          0.0      87051.0
1   debit_card    1529           572.01          0.0      40564.0
2      voucher    5689           517.62          0.0      17118.0
3  credit_card   76739           275.41          0.0     270550.0


Helps spot if payment verification or fraud checks are causing slow approvals.

Root Cause Slicing:

        Seller Fulfillment Issue (time_to_ship high) -- Time from approval to carrier pickup by seller

In [54]:
import sqlite3
import pandas as pd

# Connect to DB
conn = sqlite3.connect("olist.sqlite")

query = """

-- Time from approval to carrier pickup (per order)
WITH ship_times AS (
    SELECT 
        o.order_id,
        CAST((julianday(o.order_delivered_carrier_date) - julianday(o.order_approved_at)) * 24 AS INTEGER) AS time_to_ship_hours
    FROM orders o
    WHERE o.order_delivered_carrier_date IS NOT NULL
      AND o.order_approved_at IS NOT NULL
)
SELECT 
    COUNT(*) AS total_orders,
    ROUND(AVG(time_to_ship_hours), 2) AS avg_ship_hours,
    ROUND(MIN(time_to_ship_hours), 2) AS min_ship,
    ROUND(MAX(time_to_ship_hours), 2) AS max_ship
FROM ship_times;



"""

df = pd.read_sql_query(query, conn)
print(df)

# Save to CSV
# df.to_csv("Segment by dimension.csv", index=False)
# print("✅ Results exported to delivery_analysis.csv")

conn.close()

   total_orders  avg_ship_hours  min_ship  max_ship
0         97644           66.83   -4109.0    3018.0


Flags seller stockouts, packaging delays, or late handovers to carrier.

Root Cause Slicing:

        Last-Mile Carrier Issue (time_carrier_to_customer high)

In [56]:
import sqlite3
import pandas as pd

# Connect to DB
conn = sqlite3.connect("olist.sqlite")

query = """

-- Time from carrier pickup to delivery, segmented by customer zip prefix
WITH last_mile AS (
    SELECT 
        o.order_id,
        c.customer_zip_code_prefix,
        CAST((julianday(o.order_delivered_customer_date) - julianday(o.order_delivered_carrier_date)) * 24 AS INTEGER) AS time_carrier_to_customer_hours
    FROM orders o
    JOIN customers c ON o.customer_id = c.customer_id
    WHERE o.order_delivered_customer_date IS NOT NULL
      AND o.order_delivered_carrier_date IS NOT NULL
)
SELECT 
    customer_zip_code_prefix,
    COUNT(*) AS orders,
    ROUND(AVG(time_carrier_to_customer_hours), 2) AS avg_last_mile_hours,
    ROUND(MIN(time_carrier_to_customer_hours), 2) AS min_last_mile,
    ROUND(MAX(time_carrier_to_customer_hours), 2) AS max_last_mile
FROM last_mile
GROUP BY customer_zip_code_prefix
ORDER BY avg_last_mile_hours DESC
LIMIT 20;



"""

df = pd.read_sql_query(query, conn)
print(df)

# Save to CSV
# df.to_csv("Segment by dimension.csv", index=False)
# print("✅ Results exported to delivery_analysis.csv")

conn.close()

    customer_zip_code_prefix  orders  avg_last_mile_hours  min_last_mile  \
0                      64013       1               4353.0         4353.0   
1                      69308       1               4059.0         4059.0   
2                      29724       1               3502.0         3502.0   
3                      65735       2               2503.5         1097.0   
4                      61663       1               2393.0         2393.0   
5                      87895       1               2217.0         2217.0   
6                      29278       4               1953.0          161.0   
7                      28155       1               1855.0         1855.0   
8                      60865       2               1779.5          218.0   
9                      59970       1               1724.0         1724.0   
10                     48130       2               1708.5         1664.0   
11                     69316       1               1706.0         1706.0   
12          

Helps identify last-mile carrier issues or address accuracy problems, clustered by ZIP code.

Correlate with reviews and refunds: 

        late deliveries → negative reviews and refunds. 
        Use uplift tests: compare retention/repeat purchase rates for customers with late deliveries vs without.

In [62]:
import sqlite3
import pandas as pd

# Connect to DB
conn = sqlite3.connect("olist.sqlite")

query = """

WITH orders_flagged AS (
    SELECT 
        o.order_id,
        o.customer_id,
        CASE 
            WHEN o.order_delivered_customer_date > o.order_estimated_delivery_date THEN 1
            ELSE 0
        END AS late_flag
    FROM orders o
    WHERE o.order_delivered_customer_date IS NOT NULL
),
reviews_agg AS (
    SELECT 
        o.customer_id,
        AVG(r.review_score) AS avg_review,
        SUM(CASE WHEN r.review_score <= 3 THEN 1 ELSE 0 END) AS negative_reviews
    FROM orders_flagged o
    LEFT JOIN order_reviews r ON o.order_id = r.order_id
    GROUP BY o.customer_id
),
refunds_agg AS (
    SELECT 
        customer_id,
        SUM(CASE WHEN order_status = 'refunded' THEN 1 ELSE 0 END) AS refund_count
    FROM orders
    GROUP BY customer_id
),
repeat_agg AS (
    SELECT 
        customer_id,
        COUNT(DISTINCT order_id) AS total_orders,
        CASE WHEN COUNT(DISTINCT order_id) > 1 THEN 1 ELSE 0 END AS repeat_customer
    FROM orders
    GROUP BY customer_id
)
SELECT 
    o.late_flag,
    COUNT(DISTINCT o.customer_id) AS customers,
    ROUND(AVG(r.avg_review),2) AS avg_review,
    SUM(r.negative_reviews) AS total_negative_reviews,
    SUM(f.refund_count) AS total_refunds,
    SUM(rep.repeat_customer) * 1.0 / COUNT(*) AS repeat_rate
FROM orders_flagged o
LEFT JOIN reviews_agg r ON o.customer_id = r.customer_id
LEFT JOIN refunds_agg f ON o.customer_id = f.customer_id
LEFT JOIN repeat_agg rep ON o.customer_id = rep.customer_id
GROUP BY o.late_flag;




"""

df = pd.read_sql_query(query, conn)
print(df)

# Save to CSV
# df.to_csv("Segment by dimension.csv", index=False)
# print("✅ Results exported to delivery_analysis.csv")

conn.close()

   late_flag  customers  avg_review  total_negative_reviews  total_refunds  \
0          0      88649        4.29                   15276              0   
1          1       7827        2.57                    5036              0   

   repeat_rate  
0          0.0  
1          0.0  


What this does

        late_flag = 1 → late deliveries

        late_flag = 0 → on-time deliveries

        Computes per-group metrics:

        Avg review score

        Total negative reviews (score <= 3)

        Total refunds

        Repeat purchase rate (percentage of customers with >1 order)

This lets you perform a simple uplift analysis: compare customers with late vs on-time deliveries.

Visual diagnostics: heatmap of average delivery time by origin zip → destination zip (or seller zip → customer zip). Choropleth by state/zip p90 delays.

In [65]:
import sqlite3
import pandas as pd
import numpy as np
import plotly.express as px

# Connect to SQLite DB
conn = sqlite3.connect("olist.sqlite")

# --- 1. Heatmap: Average delivery time by seller zip → customer zip ---
query_heatmap = """
WITH delivery_times AS (
    SELECT 
        s.seller_zip_code_prefix AS origin_zip,
        c.customer_zip_code_prefix AS dest_zip,
        (julianday(o.order_delivered_customer_date) - julianday(o.order_approved_at)) * 24 AS delivery_hours
    FROM orders o
    JOIN order_items oi ON o.order_id = oi.order_id
    JOIN sellers s ON oi.seller_id = s.seller_id
    JOIN customers c ON o.customer_id = c.customer_id
    WHERE o.order_delivered_customer_date IS NOT NULL
)
SELECT 
    origin_zip,
    dest_zip,
    ROUND(AVG(delivery_hours),2) AS avg_delivery_hours,
    COUNT(*) AS orders_count
FROM delivery_times
GROUP BY origin_zip, dest_zip
LIMIT 5000;
"""
df_heatmap = pd.read_sql_query(query_heatmap, conn)

# Pivot for heatmap
heatmap_matrix = df_heatmap.pivot(index='origin_zip', columns='dest_zip', values='avg_delivery_hours')

# Plot heatmap
fig_heatmap = px.imshow(
    heatmap_matrix,
    labels=dict(x="Customer Zip", y="Seller Zip", color="Avg Delivery Hours"),
    x=heatmap_matrix.columns,
    y=heatmap_matrix.index,
    color_continuous_scale="Viridis"
)
fig_heatmap.update_layout(title="Avg Delivery Time: Seller Zip → Customer Zip")
fig_heatmap.write_html("heatmap.html")

# --- 2. Choropleth: p90 delivery delays by state ---
query_state = """
WITH delays AS (
    SELECT 
        c.customer_state,
        (julianday(o.order_delivered_customer_date) - julianday(o.order_estimated_delivery_date)) * 24 AS late_hours
    FROM orders o
    JOIN customers c ON o.customer_id = c.customer_id
    WHERE o.order_delivered_customer_date IS NOT NULL
)
SELECT customer_state, late_hours
FROM delays;
"""
df_state = pd.read_sql_query(query_state, conn)

# Compute p90 per state
df_p90 = df_state.groupby('customer_state')['late_hours'].apply(lambda x: np.percentile(x, 90)).reset_index()
df_p90.rename(columns={'late_hours': 'p90_delay_hours'}, inplace=True)

# Plot choropleth (US state codes)
fig_choropleth = px.choropleth(
    df_p90,
    locations='customer_state',
    locationmode="USA-states",
    color='p90_delay_hours',
    color_continuous_scale="Reds",
    scope="usa",
    labels={'p90_delay_hours': 'P90 Delay Hours'}
)
fig_choropleth.update_layout(title="P90 Delivery Delay by State")
fig_choropleth.write_html("choropleth.html")

# Close DB connection
conn.close()


What this does:

        Heatmap: Shows which seller→customer zip combinations are slowest.

        Choropleth: Highlights states with the worst 90th percentile delivery delays.

        Can easily be exported or embedded in a Power BI / web dashboard.

Experimentation: A/B test faster carriers or prioritized handling for specific sellers/segments and measure change in delivery times and NPS.

In [70]:

import sqlite3
import pandas as pd

# Connect to DB
conn = sqlite3.connect("olist.sqlite")

query = """

-- Define experiment groups
WITH experiment AS (
    SELECT 
        o.order_id,
        o.customer_id,
        oi.seller_id,
        o.order_delivered_customer_date,
        o.order_estimated_delivery_date,
        o.order_approved_at,
        o.order_delivered_carrier_date,
        CASE 
            WHEN oi.seller_id IN (SELECT seller_id FROM sellers) THEN 'priority_seller'
            ELSE 'control'
        END AS experiment_group
    FROM orders o
    JOIN order_items oi ON o.order_id = oi.order_id
    WHERE o.order_delivered_customer_date IS NOT NULL
),

-- Compute delivery times
delivery_metrics AS (
    SELECT 
        experiment_group,
    
        (julianday(order_delivered_customer_date) - julianday(order_approved_at)) * 24 AS time_to_ship_hours,
        (julianday(order_delivered_customer_date) - julianday(order_estimated_delivery_date)) * 24 AS late_hours
    FROM experiment
)

-- Aggregate metrics by experiment group
SELECT 
    experiment_group,
    COUNT(*) AS total_orders,
    ROUND(AVG(time_to_ship_hours),2) AS avg_time_to_ship,
    ROUND(MIN(time_to_ship_hours),2) AS min_time_to_ship,
    ROUND(MAX(time_to_ship_hours),2) AS max_time_to_ship,
    ROUND(AVG(late_hours),2) AS avg_late_hours
FROM delivery_metrics
GROUP BY experiment_group;


"""

df = pd.read_sql_query(query, conn)
print(df)

# Save to CSV
# df.to_csv("Segment by dimension.csv", index=False)
# print("✅ Results exported to delivery_analysis.csv")

conn.close()

  experiment_group  total_orders  avg_time_to_ship  min_time_to_ship  \
0  priority_seller        110196            288.82           -167.75   

   max_time_to_ship  avg_late_hours  
0           5004.02          -272.0  


Actionable interventions:

        Automate approval step where safe, reduce manual review burden.

        Enforce SLA with sellers: auto-suspend sellers with persistent shipping lateness; provide training or incentives.

        Re-route orders to alternate sellers or warehouses when seller's stock shows frequent delays.

        Work with carriers on high-delay zip codes — increase pickups/frequency or change carrier.

5) Forecast demand to reduce stockouts — approach & features

        Objectives: forecast product-level demand (daily/weekly) to:

                Set safety stock and reorder points.

                Drive warehouse replenishment planning.

Complete Pipeline for product-level demand forecasting, including feature engineering and aggregation steps, suitable for driving safety stock and reorder point calculations.

1. Aggregate daily sales per product per warehouse:

In [76]:

import sqlite3

conn = sqlite3.connect("olist.sqlite")
cursor = conn.cursor()

sql = """
CREATE TABLE IF NOT EXISTS daily_sales AS
WITH order_dates AS (
    SELECT 
        o.order_id,
        o.order_purchase_timestamp,
        o.customer_id
    FROM orders o
    WHERE o.order_status = 'delivered'
),
product_sales AS (
    SELECT 
        oi.product_id,
        s.seller_id AS warehouse_id,
        DATE(o.order_purchase_timestamp) AS sales_date,
        COUNT(oi.order_item_id) AS units_sold,
        SUM(oi.price) AS revenue
    FROM order_items oi
    JOIN order_dates o ON oi.order_id = o.order_id
    JOIN sellers s ON oi.seller_id = s.seller_id
    GROUP BY oi.product_id, warehouse_id, sales_date
)
SELECT *
FROM product_sales;


"""

# Execute DDL
cursor.execute(sql)
conn.commit()
conn.close()


In [ ]:
import pandas as pd
conn = sqlite3.connect("olist.sqlite")

df = pd.read_sql_query("SELECT * FROM daily_sales", conn)
print(df)

# Save to CSV
df.to_csv("Daily_Sales.csv", index=False)

conn.close()


Explanation:

        Aggregates daily units sold and revenue per product.

        Missing dates (no sales) should be filled with zeros later.

2️⃣ Add time-based features

In [ ]:

import sqlite3

conn = sqlite3.connect("olist.sqlite")
cursor = conn.cursor()

sql = """
-- Add day-of-week, month, week number
CREATE TABLE IF NOT EXISTS daily_sales_features AS
SELECT 
    ds.*,
    STRFTIME('%w', ds.sales_date) AS day_of_week,  -- 0=Sunday
    STRFTIME('%m', ds.sales_date) AS month,
    STRFTIME('%W', ds.sales_date) AS week_of_year
FROM daily_sales ds;

"""

# Execute DDL
cursor.execute(sql)
conn.commit()
conn.close()


In [ ]:
import pandas as pd
conn = sqlite3.connect("olist.sqlite")

df = pd.read_sql_query("SELECT * FROM daily_sales_features", conn)
print(df)

# Save to CSV
df.to_csv("Daily_Sales_features.csv", index=False)

conn.close()


Explanation:

        Helps capture seasonality and weekly patterns.

3️⃣ Add lag features and rolling statistics

In [ ]:
import sqlite3

conn = sqlite3.connect("olist.sqlite")
cursor = conn.cursor()

sql = """
-- Lag-1 units sold

CREATE TABLE IF NOT EXISTS daily_sales_lag AS
SELECT 
    f1.product_id,
    f1.warehouse_id,
    f1.sales_date,
    f1.units_sold,
    f1.revenue,
    COALESCE(f2.units_sold,0) AS lag_1_units
FROM daily_sales_features f1
LEFT JOIN daily_sales_features f2
  ON f1.product_id = f2.product_id
  AND f1.warehouse_id = f2.warehouse_id
  AND DATE(f2.sales_date, '+1 day') = f1.sales_date;


"""

# Execute DDL
cursor.execute(sql)
conn.commit()
conn.close()


In [ ]:
import pandas as pd
conn = sqlite3.connect("olist.sqlite")

df = pd.read_sql_query("SELECT * FROM daily_sales_lag", conn)
print(df)

# Save to CSV
df.to_csv("Daily_Sales_lag.csv", index=False)

conn.close()

Explanation:

        lag_1_units is the number of units sold the previous day.

        You can extend this pattern for lag_7, rolling 7-day average, rolling 30-day std using similar joins.

4️⃣ Include promotional or B2B signals:

        If using Leads_Closed or Leads_Qualified:

In [ ]:
import sqlite3

conn = sqlite3.connect("olist.sqlite")
cursor = conn.cursor()

sql = """
-- Example: flag products promoted via leads

CREATE TABLE IF NOT EXISTS sales_with_promotions AS
SELECT 
    ds.*,
    CASE WHEN lc.mql_id IS NOT NULL THEN 1 ELSE 0 END AS promo_flag
FROM daily_sales_lag ds
LEFT JOIN leads_closed lc 
  ON ds.product_id = lc.mql_id;

"""

# Execute DDL
cursor.execute(sql)
conn.commit()
conn.close()

In [ ]:
import pandas as pd
conn = sqlite3.connect("olist.sqlite")

df = pd.read_sql_query("SELECT * FROM sales_with_promotions", conn)
print(df)

# Save to CSV
df.to_csv("sales_with_promotions.csv", index=False)

conn.close()

Explanation:

        Flags days with potential promotions for demand forecasting.

5️⃣ Compute safety stock and reorder points:

        Assuming you know lead time (LT) in days and target service level z:

Define a user-defined function (UDF) in SQLite

In [ ]:
import sqlite3
import pandas as pd

# Connect
conn = sqlite3.connect("olist.sqlite")

# Load daily_sales into pandas
df = pd.read_sql_query("SELECT * FROM daily_sales", conn)

# Compute mean and std per product + warehouse
demand_stats = df.groupby(['product_id', 'warehouse_id'])['units_sold'].agg(
    avg_daily_demand='mean',
    std_daily_demand='std'
).reset_index()

# Optional: save back to SQLite
demand_stats.to_sql("demand_stats", conn, if_exists="replace", index=False)

print(demand_stats.head())
conn.close()


Safety stock and reorder point calculation:

In [93]:
import sqlite3

conn = sqlite3.connect("olist.sqlite")
cursor = conn.cursor()

sql = """
-- Example: Safety stock = z * σ * sqrt(LT), Reorder point = mean*LT + SS
-- Assuming z=1.65 (95% service level), LT in days = 7

CREATE TABLE IF NOT EXISTS reorder_points AS
SELECT 
    ds.product_id,
    ds.warehouse_id,
    avg_daily_demand,
    std_daily_demand,
    ROUND(1.65 * std_daily_demand * SQRT(7),0) AS safety_stock,
    ROUND(avg_daily_demand * 7 + 1.65 * std_daily_demand * SQRT(7),0) AS reorder_point
FROM demand_stats ds;
"""

# Execute DDL
cursor.execute(sql)
conn.commit()
conn.close()

Explanation:

        safety_stock accounts for demand variability during lead time.

        reorder_point triggers replenishment when inventory reaches this level.

In [ ]:
# Useful if individual product data is sparse.

# You can forecast at category × region and then distribute to individual products.

import sqlite3

conn = sqlite3.connect("olist.sqlite")
cursor = conn.cursor()

sql = """
-- Aggregate by product category and seller region

CREATE TABLE IF NOT EXISTS category_region_sales AS
SELECT 
    p.product_category_name,
    s.seller_state AS region,
    DATE(o.order_purchase_timestamp) AS sales_date,
    SUM(oi.order_item_id) AS units_sold
FROM order_items oi
JOIN orders o ON oi.order_id = o.order_id
JOIN products p ON oi.product_id = p.product_id
JOIN sellers s ON oi.seller_id = s.seller_id
WHERE o.order_delivered_customer_date IS NOT NULL
GROUP BY product_category_name, region, sales_date;

"""

# Execute DDL
cursor.execute(sql)
conn.commit()
conn.close()

Forecasting: export prepared tables to Python / R / ML frameworks:

        Prophet: seasonality & holidays

        SARIMA: stationary time series

        XGBoost/LightGBM: cross-sectional ML